Imports

In [47]:
import sqlite3
import pandas as pd

Data Preparation

In [62]:
df = pd.read_csv("/workspaces/swiss-tourism-insights/data/hotels_list.csv")

# Drop duplicate rows
df.drop_duplicates(inplace=True)
print(f"Removed duplicates. New shape: {df.shape}")

#Check for missing values
df.isnull().sum()

#Drop rows with critical missing data
df = df.dropna(subset=["hotel", "price_per_night"])

# First few rows
df.head()

Removed duplicates. New shape: (979, 5)


,hotel,place,price_per_night,rating,review_counter
0,Hard Rock Hotel Davos,Davos,€ 225,"8,4",1.653 Bewertungen
1,Good Night Inn,Brig,€ 138,"8,2",2.669 Bewertungen
2,Agriturismo Regusci,Camorino,€ 165,"8,7",765 Bewertungen
3,Ustreia Orta,Lohn,€ 169,"9,6",52 Bewertungen
4,Hotel Drei Könige,Einsiedeln,€ 177,"8,6",948 Bewertungen


In [70]:
#Convert Data Types
df["price_cleaned"] = df["price_per_night"].str.extract(r"(\d+)").astype(float)  # Adjust column name if needed

# Clean the review_counter column
df["review_counter_cleaned"] = df["review_counter"].str.extract(r"([\d\.]+)")
df["review_counter_cleaned"] = df["review_counter_cleaned"].str.replace(".", "", regex=False)
df["review_counter_cleaned"] = pd.to_numeric(df["review_counter_cleaned"], errors="coerce").astype("Int64")

df["rating_cleaned"] = df["rating"].astype(str).str.replace(",", ".", regex=False).astype(float)

# Extract the part after the comma in the 'place' column, if available
df["place_cleaned"] = df["place"].str.extract(r",\s*(.+)$")  # Extract everything after the comma

# Fill missing values (rows without a comma) with the original place
df["place_cleaned"] = df["place_cleaned"].fillna(df["place"])

df.head()

,hotel,place,price_per_night,rating,review_counter,price_cleaned,review_counter_cleaned,rating_cleaned,place_cleaned
0,Hard Rock Hotel Davos,Davos,€ 225,"8,4",1.653 Bewertungen,225.0,1653,8.4,Davos
1,Good Night Inn,Brig,€ 138,"8,2",2.669 Bewertungen,138.0,2669,8.2,Brig
2,Agriturismo Regusci,Camorino,€ 165,"8,7",765 Bewertungen,165.0,765,8.7,Camorino
3,Ustreia Orta,Lohn,€ 169,"9,6",52 Bewertungen,169.0,52,9.6,Lohn
4,Hotel Drei Könige,Einsiedeln,€ 177,"8,6",948 Bewertungen,177.0,948,8.6,Einsiedeln


Load data to SQLite Database

In [77]:
# Load the CSV with semicolon as separator
places_df = pd.read_csv('../data/swiss_places.csv', sep=';')

# Create SQLite DB in project root folder
conn = sqlite3.connect('../swiss_travel.db')

# Save tables to DB
df.to_sql('hotels', conn, if_exists='replace', index=False)
places_df.to_sql('places', conn, if_exists='replace', index=False)

print("Data loaded into SQLite!")

Data loaded into SQLite!
